# Lineage Tree comparison, simulate trees to test HSA program

## 1 load required packages

In [1]:
library(data.tree)

Warning message:
"package 'data.tree' was built under R version 3.4.3"

In [2]:
library(parallel)

## 2 functions to use

### 2.1 sigmoid function, F.sigmoid(x,a=100). 
Help determine whether a gene is on or off.

$f(x) = 2/(1 + e^{-ax}) -1$
* x: input value
* a: activation constant. default: 100

In [3]:
F.sigmoid <- function(x, a=100){
  return (2 / (1 + exp(-a*x)) -1)
}

### 2.2 gene.level to gene.OF

F.geneOF(gene.level)

gene.level is a vector with length N. gene.level = c(s1, s2, ..., si, ..., sN), where -1<= xi <= 1. 

return a gene.OF vector of the same length. Gene *i* is ON when 0< si <=1, and OFF when -1<= si <=0. 1 for ON and 0 for OFF. 

 F.geneOF(c(0.1, 1, -0.5, 0)). Return c(1,1,0,0). 
* N: number of genes. 

In [4]:
F.geneOF <- function(gene.level){
    if (any(gene.level > 1) || any(gene.level < -1)){
        print("warning! some gene.level is outside the value limit of -1<= gene.level <= 1")
    }
    return (as.numeric(gene.level > 0))
}

### 2.3 cell divide. F.divide(gene.level)
gene.level is a vector with length N. gene.level = c(s1, s2, ..., si, ..., sN), where -1<= xi <= 1

return a data.frame with two columns, L and R, for the two offsprings. 
* N: number of genes. 
* Gene 1 is a cell-cycle regulator. If gene 1 is ON, the cell divides instantaneously, and gene 1 is turned OFF in both daughter cells. 
* Gene 2 is a asymmetric cell division gene. When gene 1 is ON:
    * if gene 2 is ON, the cell divides asymmetrically. The product of gene 2 segregates asymmetrically among the daughter cells: it is turned OFF in the left daughter, S2L = -1, and stays ON in the right daughter cell, at the same expression level.
    * if gene 2 is OFF, then the cell divides aymmetrically and both daughter cells retain the gene expression pattern of the mother cell.
    
Example:
* (0.9, 0.8, -0.8, 1) -> (-1, -1, -0.8, 1), (-1, 1, -0.8, 1). Divide asymmetrically.
* (-0.9, 0.8, -0.8, 1) -> NULL. Do not divide.
* (0.9, -0.8, -0.8, 1) -> (-1, -0.8, -0.8, 1), (-1, -0.8, -0.8, 1). Divide symmetrically.

In [5]:
F.divide <- function(gene.level){
    if (gene.level[1] > 0){
        L <-  gene.level
        R <- gene.level
        L[1] <- -1
        R[1] <- -1
        if (gene.level[2] >0) L[2] <- -1
        return (data.frame(L,R))
    }
    return (NULL)
}

### 2.4 cell grow

The gene expression pattern in a cell at time t during development is represented by a state vector S(t), whose elements $s_{i}(t)$ described the expression states of genes i=1, 2, ..., N. $-1 <= s_{i}(t) <= 1$.

$s_{i}(t+1) = f [\sum_{j=1}^{N}r_{ij}s_{j}(t)]$

where $f(x) = 2/(1 + e^{-ax}) -1$, as defined by F.sigmoid.

R is the matrix of $r_{ij}$, which represents gene interaction network (**gene.network**). Given a S(t), calculate S(t+1).

For example. 

$R = \begin {vmatrix} 
            0 & 0 & 1.7 & 0 \\ 
            -0.3 & -0.9 & 0 & -0.8 \\  
            -1.2 & 0.7 & 0.2 & 0 \\
            0 & -0.8 & 0 & -0.8 \\
            \end {vmatrix}$

$S(t) = \begin {vmatrix} 1 \\ 1\\ 1\\ 1\\ \end {vmatrix}$. After **cell divide**, two daughter cells are born, which are $S_{L}(t)=\begin {vmatrix} -1 \\ -1\\ 1\\ 1\\ \end {vmatrix}$ and $S_{R}(t)=\begin {vmatrix} -1 \\ 1\\ 1\\ 1\\ \end {vmatrix}$. 

At time **t+1**, $S_{L}(t+1)=f(R*S_{L}(t))$

**F.grow(gene.level, gene.network, a=100); a; activation constant.**

In [6]:
F.grow <- function(gene.level, gene.network, a = 100){
    return (F.sigmoid(as.vector(gene.network %*% gene.level), a))
}

### 2.5 generate random gene interaction network (gene.network)

gene.network is a N\*N matrix. Of N\*N values, N\*K are normal random variates.
* N: number of genes
* K: average in-degree, which means average interations per gene.
* digits: number of digits to round after the decimal point. No rounding under default setting;

F.randNetwork(N,K,digits = NULL)

In [7]:
F.randNetwork <- function(N,K,digits = NULL){
    gene.interactions = rnorm(N*K)
    gene.network = c(gene.interactions, rep(0, N*(N-K)))
    if (is.null(digits))
        return (matrix(sample(gene.network), ncol = N, nrow = N))
    else
        return (round((matrix(sample(gene.network), ncol = N, nrow = N)), digits))
}

### 2.6 generate random initial gene expression status (gene.init)

An initial gene expression pattern, S(0), is created by setting $s_{1} = -1$ and randomly setting each $s_{i}(0)$ for i = 2, 3, ..., N to either -1 or 1

F.randInit(N)
* N number of genes
* F.randInt(4), return c(-1, (-1 or 1), (-1 or 1), (-1 or 1)).

In [8]:
F.randInit <- function(N){
    randInit <- c(-1,sample(c(1,-1), size = N-1, replace = TRUE))
    return (randInit)
}

### 2.7 develop the tree based on a given gene.init and gene.network

return a data.tree variable.

F.develop(gene.init, gene.network, tmax = 50, Dmax = 10)

Each node with five fields, which are DAge, tAge, Lineage, gene.level, gene.OF.

Use a simplified simmulation method, not the one used in the paper (Rolf Lohaus *et al.*, 2007). 
* In the paper
    * Development stops when all the cells have differentiated or the first undifferentiated cell have completed Dmax rounds of cell division.
    * Gene expression pattern were calculated differently for undifferentiated and differentiated cells.
* Here
    * Development stops when the first undifferentiated cell have completed Dmax rounds of cell divison, or development time reached tmax.
    * Gene expression level were the same as generated when the program ends.

The cells presentat the end of development are defined as terminal.

max tAge is tmax, and max DAge is Dmax. tAge: develop time. DAge: division times.

In [9]:
F.develop <- function(gene.init, gene.network, tmax = 50, Dmax = 10, a = 100){
    gene.OF <- F.geneOF(gene.init)
    cell.tree  <- Node$new(name = "Root", DAge = 0, tAge = 0, Lineage="", Gene.level = gene.init, Gene.OF = gene.OF)
    tAge = 0
    DAge = 0
    while (TRUE){
        if (tAge > tmax || DAge > Dmax) break
        for (cell.leaf in cell.tree$leaves){
            cell.divide = F.divide(cell.leaf$Gene.level)
            if (is.null(cell.divide)){
                tAge  <- cell.leaf$tAge + 1
                if (tAge > tmax || DAge > Dmax) break
                name  <- "LR"
                DAge <- cell.leaf$DAge
                Lineage <- cell.leaf$Lineage
                Gene.level <- F.grow(cell.leaf$Gene.level, gene.network = gene.network, a = a)
                Gene.OF <- F.geneOF(Gene.level)
#                 cell.leaf$AddChild(name = name, DAge = DAge, tAge = tAge, 
#                                    Lineage = Lineage, Gene.level = Gene.level, Gene.OF = Gene.OF)
                
#                 cell.leaf$name <- name
                cell.leaf$DAge <- DAge
                cell.leaf$tAge <- tAge
                cell.leaf$Lineage <- Lineage
                cell.leaf$Gene.level <- Gene.level
                cell.leaf$Gene.OF <- Gene.OF
            }
            else {
                tAge <- cell.leaf$tAge + 1
                DAge <- cell.leaf$DAge + 1
                if (tAge > tmax || DAge > Dmax) break
                Lname <- "L"
                LLineage <- paste0(cell.leaf$Lineage,"0")
                Rname <- "R"
                RLineage <- paste0(cell.leaf$Lineage,"1")
                LRinit <- F.divide(cell.leaf$Gene.level)
                LGene.level <- F.grow(LRinit$L, gene.network = gene.network, a = a)
                RGene.level <- F.grow(LRinit$R, gene.network = gene.network, a = a)
                LGene.OF  <- F.geneOF(LGene.level)
                RGene.OF <- F.geneOF(RGene.level)
                cell.leaf$AddChild(name = Lname, DAge = DAge, tAge = tAge, 
                                   Lineage = LLineage, Gene.level = LGene.level, Gene.OF = LGene.OF)
                cell.leaf$AddChild(name = Rname, DAge = DAge, tAge = tAge, 
                                   Lineage = RLineage, Gene.level = RGene.level, Gene.OF = RGene.OF)
            }
        }
    }
    return (cell.tree)
}

### 2.8 Save the Random Tree Generated by F.develop

F.saveRandomTree(cell.tree, score = "common",class = "Gene.OF",outfilePrefix = "")

Four files will be written.
* "cell.tree.all": data.frame of cell.tree
* "cell.tree.network": interaction network of genes.
* "cell.tree.leaves": text file to be used by HSA program. Lineage is the Lineage in cell.tree. 
    * Name is the name. Class is the "Gene.OF", unless other values were assigned.
* "cell.tree.score": text file to be used by HSA program. 
    * three columns. treeS cell class, treeT cell class, score (integer).
    * If score == "common", the score for two cell classes is based on the number of common "ON"/"OFF" genes. 1011,1010, score 3.
    * If score == "same", the score is 2 if cell classes the same or 0 if not.
    * If score == "norm", the average score will be 1. score from the "common" method.   
        calculate z-score. round (z-score+1) to integer. 
        (0,4,16) will become (0.2, 0.7, 2.1), then round to (0,1,2). (sd caluclated by R, denominator n-1)

In [10]:
F.saveRandomTree <- function(cell.tree, network = NULL, score = "common", 
                             class = "Gene.OF", outfilePrefix = "",silent = FALSE) {
    cell.tree.dfAll <- ToDataFrameTree(cell.tree,"pathString","level","name",'Lineage', 'DAge','tAge',
                                     Gene.OF = function(node) paste0(node$Gene.OF, collapse = ''),
                                     Gene.level = function(node) paste0(node$Gene.level, collapse = ';'),
                                     isLeaf = function(node) node$isLeaf)
    outfile.cell.tree.all <- paste0(outfilePrefix, "cell.tree.all")
    write.table(cell.tree.dfAll, outfile.cell.tree.all, sep="\t", row.names = FALSE)
    if (!silent) print(paste0("write the all details for the tree to file ", outfile.cell.tree.all))
    
    cell.tree.dfLeaf <- ToDataFrameTable(cell.tree,"pathString","level","name",'Lineage', 'DAge','tAge',
                                         Gene.OF = function(node) paste0(node$Gene.OF, collapse = ''))
    cell.tree.Leaf <- cell.tree.dfLeaf[,c("Lineage","name","Gene.OF")]
    outfile.cell.tree.leaves <- paste0(outfilePrefix, "cell.tree.leaves")
    write.table(cell.tree.Leaf, outfile.cell.tree.leaves,sep = "\t", row.names = FALSE, quote = FALSE)
    #remove the last newline symbol in outfile.cell.tree.leaves
    txt <- readChar(outfile.cell.tree.leaves,file.info(outfile.cell.tree.leaves)$size,useByte = TRUE)
    txt <- gsub("\r","",txt)
    if (substr(txt,nchar(txt),nchar(txt)) == "\n")
        cat(substr(txt,1,nchar(txt)-1), file = outfile.cell.tree.leaves)
    if (!silent) print(paste0("write the leaves file for HSA to ", outfile.cell.tree.leaves))
    
    outfile.cell.tree.score <- paste0(outfilePrefix, "cell.tree.score")
    gene.OF <- cell.tree.dfLeaf$Gene.OF
    if (length(unique(gene.OF)) > 2)
        gene.OF.pairs <- combn(unique(gene.OF),2)
    else
        gene.OF.pairs <- NULL
    dfscore <- data.frame(S=c(unique(gene.OF),gene.OF.pairs[1,]), T=c(unique(gene.OF),gene.OF.pairs[2,]))
    if (score == "common")
        dfscore$Score <- apply(dfscore,1,function(x) sum(unlist(strsplit(x["S"],"")) == unlist(strsplit(x["T"],""))))
    else if (score == "same")
        dfscore$Score <- (dfscore$S == dfscore$T)*2
    else if (score == "norm"){
        dfscore$Score <- apply(dfscore,1,function(x) sum(unlist(strsplit(x["S"],"")) == unlist(strsplit(x["T"],""))))
        dfscore$Score <- round((dfscore$Score - mean(dfscore$Score))/sd(dfscore$Score)) + 1
    } else if (!silent) print("invalid value for score, score can be same, common or norm")
        
    write.table(dfscore, outfile.cell.tree.score, sep = "\t", row.names = FALSE, col.names = FALSE, quote = FALSE)
    if (!silent) print(paste0("write the score file to ", outfile.cell.tree.score))
    
    if (!is.null(network)) {
        outfile.network  <- paste0(outfilePrefix, "cell.tree.network")
        write.table(network, outfile.network, sep = "\t", row.names = FALSE, col.names = FALSE, quote = FALSE)
        print(paste0("write the interaction network to file ", outfile.network))
    }
    else if (!silent) print("You need to provide Network in order to save it!")
}

### 2.9 generate a tree with a, N, K, Dmax and tmax

Given a Dmax and tmax, return list with **gene.init, gene.network, cell.tree**.

Use generate the randome gene interacton network and initial gene expression status

In [11]:
F.generateRandTree <- function(a, N, K, Dmax, tmax, digits = 3){
    gene.init  <- F.randInit(N)
    gene.network <- F.randNetwork(N, K, digits = digits)
    cell.tree <- F.develop(gene.init, gene.network, tmax = tmax, Dmax = Dmax, a = a)
    return(list(gene.init = gene.init, gene.network = gene.network, cell.tree = cell.tree))
}

### 2.10 count cell types based on gene.OF for a cell.tree

In [12]:
F.treeCellType <- function(tree1){
    gene.OF <- lapply(tree1$leaves, function(node) paste0(node$Gene.OF,collapse = ""))
    return(length(unique(unlist(gene.OF))))
}

### 2.11 helper function to filter the generated trees

the same as function F.generateRandTree.  
return normal result if leafTypeCount >= leafTypeMin and leafCount >= leafCountMin.  
add the field "leafCount" and "leafTypeCount".  
else, try again (total trying times <= tryMax), until leafTypeCount >= leafTypeMin and leafCout >= leafCountMin.   
else, return NULL

In [13]:
F.generateRandTreeWithFilter  <- function(a, N, K, Dmax, tmax, digits = 3, 
                                          leafTypeMin = 6, leafCountMin = 500, tryMax = 100){
    for (i in 1:tryMax){
        celltree <- F.generateRandTree(a = a, N = N, K=K, Dmax = Dmax, tmax = tmax, digits = digits)
        leafTypeCount <- F.treeCellType(celltree[["cell.tree"]])
        leafCount <- celltree[["cell.tree"]]$leafCount
        celltree$leafTypeCount <- leafTypeCount
        celltree$leafCount <- leafCount
        if (leafCount >= leafCountMin && leafTypeCount >= leafTypeMin){
            return(celltree)
        }
    }
    print(paste0("tried ", tryMax, " times. Still No good. return NULL"))
    return(NULL)
}

## 3 Generate Some Random Tree

### 3.1 set up some costant values
* N: number of genes
* K: average in-degree, which means average interations per gene.
* Dmax: max rounds of cell division
* a: activation constant
* tmax: critical period of time used to establish whether a cell divides or differentiates

In [14]:
N = 8
K = 4
Dmax = 10
tmax = 50
a = 100

### 3.2 test some function

In [15]:
gene.init <-  F.randInit(N)
gene.network <- F.randNetwork(N,K)
gene.OF <- F.geneOF(gene.init)
cell.tree  <- Node$new(name = "Root", DAge = 0, tAge = 0, Lineage="1", Gene.level = gene.init, Gene.OF = gene.OF)

In [16]:
for (cell.leaf in cell.tree$leaves) print(cell.leaf)

  levelName
1      Root


In [17]:
gene.network

-3.1954806,0.00000000,0.5032026,-1.1328098,-0.7284462,0.2098176,0.0000000,0.00000000
-0.8196165,0.09483462,-0.3003173,-0.3271960,0.0000000,1.2279651,-0.3345758,-0.23817403
0.0000000,0.00000000,0.0000000,0.0000000,0.0000000,0.0000000,-2.2967978,-2.03706139
0.0000000,0.00000000,0.0000000,0.5442188,2.3236564,0.0000000,0.0000000,1.00036413
0.6271605,1.40670111,0.0000000,1.1659624,-0.6417234,-0.9287871,-0.8501125,0.00000000
0.0000000,-0.39291016,0.0000000,0.0000000,0.0000000,0.0000000,0.9945757,0.06981529
-0.4398823,0.08964650,0.0000000,-1.3721335,0.0000000,0.0000000,0.1552225,0.00000000
0.0000000,0.00000000,0.0000000,1.3676845,0.0000000,-1.7279009,0.0000000,0.00000000


In [18]:
gene.init

[1] -1  1 -1 -1  1  1  1 -1

### 3.3 generate one Randome Tree and view

In [19]:
gene.init <-  F.randInit(N)
gene.network <- F.randNetwork(N,K)
tree1 <- F.develop(gene.init = gene.init, gene.network = gene.network,tmax = 10, Dmax = 5)
print(tree1,"Lineage","DAge","tAge",Gene.OF = function(node) paste0(node$Gene.OF, collapse = ''))
tree1$leafCount

  levelName Lineage DAge tAge  Gene.OF
1     Root             0    2 10011010
2      |--L       0    1   10 01101011
3      °--R       1    1   10 01101011


[1] 2

### 3.4 test the function of ToDataFrameTree

In [20]:
ToDataFrameTree(tree1,"pathString","level","name",'Lineage', 'DAge','tAge',
                 Gene.OF = function(node) paste0(node$Gene.OF, collapse = ''),
                 Gene.level = function(node) paste0(node$Gene.level, collapse = ';'),
                 isLeaf = function(node) node$isLeaf)

levelName,pathString,level,name,Lineage,DAge,tAge,Gene.OF,Gene.level,isLeaf
Root,Root,1,Root,,0,2,10011010,1;-1;-1;0.999107794099984;1;-1;1;-1,FALSE
|--L,Root/L,2,L,0,1,10,01101011,-1;1;1;-1;1;-0.99999379408902;1;1,TRUE
°--R,Root/R,2,R,1,1,10,01101011,-1;1;1;-1;1;-0.99999379408902;1;1,TRUE


In [21]:
tree1.df <- ToDataFrameTree(tree1,"pathString","level","name",'Lineage', 'DAge','tAge',
                 Gene.OF = function(node) paste0(node$Gene.OF, collapse = ''),
                 Gene.level = function(node) paste0(node$Gene.level, collapse = ';'),
                 isLeaf = function(node) node$isLeaf)

In [22]:
getwd()
# write.table(tree1.df,"test.cell.tree.all",sep = "\t",row.names = FALSE)

[1] "C:/Users/ATPs/Documents/GitHub/XCProject/YangLab"

### 3.5 test the function of ToDataFrameTable

only the leaves were saved in this case

In [23]:
ToDataFrameTable(tree1,"pathString","level","name",'Lineage', 'DAge','tAge',
                 Gene.OF = function(node) paste0(node$Gene.OF, collapse = ''))

pathString,level,name,Lineage,DAge,tAge,Gene.OF
Root/L,2,L,0,1,10,01101011
Root/R,2,R,1,1,10,01101011


### 3.5 count cell types

In [24]:
F.treeCellType(tree1)

[1] 1

### 3.6 generate a data.frame for HSA program

In [25]:
tree.dfLeaf <- ToDataFrameTable(tree1,"pathString","level","name",'Lineage', 'DAge','tAge',
                 Gene.OF = function(node) paste0(node$Gene.OF, collapse = ''))
tree.dfLeaf[,c("Lineage","name","Gene.OF")]

Lineage,name,Gene.OF
0,L,01101011
1,R,01101011


In [26]:
# F.saveRandomTree(tree1, network = gene.network,score = "same")

In [27]:
tree1

  levelName
1     Root 
2      |--L
3      °--R

## 4 simulate trees with Dmax 12

simulate 7 groups of lineage trees, with the following setting

a   | N   |K
--  | --  | --
1   | 16  | 4
10  | 16  | 4
100 | 16  | 2
100 | 16  | 4
100 | 16  | 8
100 | 8   | 4
100 | 32  | 4

* a: activation constant
* N: number of genes
* K: average in-degree, which means average interations per gene.

Dmax: max dividing times. Dmax = 12 (Dividing times <= Dmax)  
tmax = 50 (maximum developint times. t <= tmax).  
leafTypesMin = 6
leafCountMin = 500

leafTypeMin, the minimum types of leaves  
leafCountMin, the minmum count of leaves

### 4.1 set up the parameters

In [28]:
df.settingD12 = data.frame(a = c(1,10,100,100,100,100,100), 
                        N = c(16,16,16,16,16,8,32),
                        K = c(4,4,2,4,8,4,4),
                        Dmax = rep(12,7),
                        tmax = rep(50,7))

In [29]:
df.settingD12

a,N,K,Dmax,tmax
1,16,4,12,50
10,16,4,12,50
100,16,2,12,50
100,16,4,12,50
100,16,8,12,50
100,8,4,12,50
100,32,4,12,50


In [30]:
leafTypeMin = 6
leafCountMin = 500

### 4.2 test one

In [31]:
s <- df.settingD12[1,]
s

a,N,K,Dmax,tmax
1,16,4,12,50


In [32]:
celltrees = list()
for (i in row.names(df.settingD12)) {
    s <- df.settingD12[i,]
    celltrees[[1]] = F.generateRandTree(a = s$a, N = s$N, K = s$K, Dmax = s$Dmax, tmax = s$tmax, digits = 4)
    print(s)
    break
}
celltrees[[1]][["cell.tree"]]$leafCount


  a  N K Dmax tmax
1 1 16 4   12   50


[1] 1152

In [33]:
celltrees[[1]][["cell.tree"]]$Gene.OF

[1] 1 1 1 1 0 0 1 1 1 0 1 1 0 0 0 0

In [34]:
F.treeCellType(celltrees[[1]][["cell.tree"]])

[1] 4

In [35]:
tree1 = celltrees[[1]][["cell.tree"]]

### 4.3 generate a list to store the result

However, the memory consumption is too big. only save the necessary parts, and save the tree files directly to drive.

In [36]:
celltrees <- lapply(1:2,
                    function(x)F.generateRandTreeWithFilter(a = s$a, N = s$N, K = s$K, Dmax = s$Dmax, 
                                        tmax = s$tmax, digits = 4,
                                        leafTypeMin = leafTypeMin,
                                        leafCountMin = leafCountMin, tryMax = 100))

In [37]:
for (tr in celltrees) print(c(tr$leafCount, tr$leafTypeCount))

[1] 2466   42
[1] 4096   21


In [38]:
tr <- celltrees[[1]]
# F.saveRandomTree(cell.tree = tr$cell.tree, 
#                  score = "norm", outfilePrefix = "X:\\YangLab\\2018simulation\\", silent = TRUE)



In [39]:
# fileTest <- "C:\\Users\\ATPs\\Documents\\2018simulation\\a1N16K4\\test.txt"
# fileTestWrite <- file(fileTest,"w")
# writeLines(text = c(paste0("ID=1;a=1;N=16;K=4;L=",tr$leafCount,";T=",tr$leafTypeCount),
#                    paste(tr$gene.init,collapse = " "),
#                    paste(as.vector(tr$gene.network), collapse = " ")), con = fileTestWrite)
# close(fileTestWrite)

In [40]:
tr$leafTypeCount

[1] 42

### 4.4 save 1000 files for a=1 N=16 K=4

In [42]:
s <- df.settingD12[1,]#change 1 to 2,3,4,5,6,7 to simulate other cases

In [43]:
folder <- paste0("X:\\YangLab\\2018simulation\\a",s$a,"N",s$N,"K",s$K,"\\")

In [ ]:
fileInfo <- paste0("X:\\YangLab\\2018simulation\\a",s$a,"N",s$N,"K",s$K,".info")
fileInfoWrite <- file(fileInfo,"w")
for (n in 1:1000){
    tr <- F.generateRandTreeWithFilter(a = s$a, N = s$N, K = s$K, Dmax = s$Dmax, 
                                        tmax = s$tmax, digits = 4,
                                        leafTypeMin = leafTypeMin,
                                        leafCountMin = leafCountMin, tryMax = 100)
    filenamePrefix <- paste0(folder,"a",s$a,"N",s$N,"K",s$K,"ID",n)
    F.saveRandomTree(cell.tree = tr$cell.tree, network = NULL, score = "norm", class = "Gene.OF", 
                     outfilePrefix = filenamePrefix, silent = TRUE)
    writeLines(text = c(paste0("ID=",n,";a=",s$a,";N=",s$N,";K=",s$K,";L=",tr$leafCount,";T=",tr$leafTypeCount),
                   paste(tr$gene.init,collapse = " "),
                   paste(as.vector(tr$gene.network), collapse = " ")), con = fileInfoWrite)
}
close(fileInfoWrite)

### the tree simulation finished by run individual Rscript processes. 

Finished outside this notebook

## 5 simulate trees with Dmax 6

### 5.1 set up the parameters

In [53]:
df.settingD6 = data.frame(a = c(1,10,100,100,100,100,100), 
                        N = c(16,16,16,16,16,8,32),
                        K = c(4,4,2,4,8,4,4),
                        Dmax = rep(6,7),
                        tmax = rep(50,7))
df.settingD6

a,N,K,Dmax,tmax
1,16,4,6,50
10,16,4,6,50
100,16,2,6,50
100,16,4,6,50
100,16,8,6,50
100,8,4,6,50
100,32,4,6,50


In [54]:
leafTypeMin = 4
leafCountMin = 16

### 5.2 test One

It seems that HSA cannot run with large files. test small ones.

In [73]:
trTest <- F.generateRandTreeWithFilter(a = 1, N = 16, K = 4, Dmax = 6, 
                                        tmax = 50, digits = 4,
                                        leafTypeMin = 6,
                                        leafCountMin = 20, tryMax = 100)

In [74]:
F.saveRandomTree(cell.tree = trTest$cell.tree, 
                 score = "norm", outfilePrefix = "X:\\YangLab\\2018simulation\\", silent = TRUE)



### 5.3 simulate 1000 files for each condition

#### create folders

In [55]:
for (i in 1:7){
    s <- df.settingD6[i,]
    folder <- paste0("X:\\YangLab\\2018simulation\\a",s$a,"N",s$N,"K",s$K,"D",s$Dmax,"\\")
    dir.create(folder)
}

#### simulation of cell trees

In [57]:
for (i in 1:7){
    s <- df.settingD6[i,]
    folder <- paste0("X:\\YangLab\\2018simulation\\a",s$a,"N",s$N,"K",s$K,"D",s$Dmax,"\\")
    fileInfo <- paste0("X:\\YangLab\\2018simulation\\a",s$a,"N",s$N,"K",s$K,"D",s$Dmax,".info")
    fileInfoWrite <- file(fileInfo,"w")
    for (n in 1:1000){
        tr <- F.generateRandTreeWithFilter(a = s$a, N = s$N, K = s$K, Dmax = s$Dmax, 
                                            tmax = s$tmax, digits = 4,
                                            leafTypeMin = leafTypeMin,
                                            leafCountMin = leafCountMin, tryMax = 100)
        filenamePrefix <- paste0(folder,"a",s$a,"N",s$N,"K",s$K,"D",s$Dmax,"ID",n)
        F.saveRandomTree(cell.tree = tr$cell.tree, network = NULL, score = "norm", class = "Gene.OF", 
                         outfilePrefix = filenamePrefix, silent = TRUE)
        writeLines(text = c(paste0("ID=",n,";a=",s$a,";N=",s$N,";K=",s$K,";L=",tr$leafCount,";T=",tr$leafTypeCount),
                       paste(tr$gene.init,collapse = " "),
                       paste(as.vector(tr$gene.network), collapse = " ")), con = fileInfoWrite)
    }
    close(fileInfoWrite)
}